In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Sequential, Model
from keras.layers import Layer, Input, Dense, Dropout, Flatten, Reshape, Embedding, Concatenate, Dot, BatchNormalization, LeakyReLU
from keras.optimizers import Adam
from keras.losses import MeanSquaredError, BinaryCrossentropy
from sklearn.model_selection import train_test_split

2024-05-13 12:46:13.243888: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 12:46:13.245807: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 12:46:13.249236: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-13 12:46:13.298707: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 12:46:14.322253: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

In [3]:
from config import Config
from utils.data_loader import DataLoader

In [4]:
df = DataLoader().load_train_data()

In [5]:
df

,user_id,item_id,rating
0,716,836,4
1,620,422,1
2,547,289,3
3,758,385,4
4,627,197,5
...,...,...,...
79995,336,628,3
79996,749,637,1
79997,758,313,4
79998,380,139,1


943

In [10]:
x_train, x_test, y_train, y_test = train_test_split(df[df.columns[:-1]].values, df[df.columns[-1]].values, test_size=0.2, random_state=40)

In [11]:
latent_dim = 5

In [12]:
item_input = Input(shape=[1], name='item_input')
item_embedding = Embedding(num_items + 1, latent_dim, name='item_embedding')(item_input)
item_flatten = Flatten(name='item_flatten')(item_embedding)

2024-05-12 16:26:30.258551: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-12 16:26:30.259342: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [13]:
user_input = Input(shape=[1], name='user_input')
user_embedding = Embedding(num_users + 1, latent_dim, name='user_embedding')(user_input)
user_flatten = Flatten(name='user_flatten')(user_embedding)

In [ ]:
class DeepMF(Model):
    
    

In [14]:
rating_vec = Dot(axes=1, name='rating_dot')([item_flatten, user_flatten])

In [15]:
deepmf = Model([user_input, item_input], rating_vec)

In [16]:
deepmf.compile(optimizer=Adam(), metrics=['mae'], loss=MeanSquaredError())

In [17]:
deepmf.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ item_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_input          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_embedding      │ (None, 1, 5)      │      8,415 │ item_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_embedding      │ (None, 1, 5)      │      4,720 │ user_input[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_flatten        │ (None, 5)         │          0 │ item_embedding[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_flatten        │ (None, 5)         │          0 │ user_embedding[0… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rating_dot (Dot)    │ (None, 1)         │          0 │ item_flatten[0][… │
│                     │                   │            │ user_flatten[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 13,135 (51.31 KB)

 Trainable params: 13,135 (51.31 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
deepmf.fit(
    [x_train[:, 0], x_train[:, 1]],
    y_train,
    validation_data=([x_test[:, 0], x_test[:, 1]], y_test),
    epochs=10
)

Epoch 1/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 757us/step - loss: 13.0422 - mae: 3.4242 - val_loss: 5.6572 - val_mae: 2.0962
Epoch 2/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 615us/step - loss: 4.0214 - mae: 1.6792 - val_loss: 1.8775 - val_mae: 1.0737
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 612us/step - loss: 1.5946 - mae: 0.9889 - val_loss: 1.2523 - val_mae: 0.8672
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 592us/step - loss: 1.1327 - mae: 0.8312 - val_loss: 1.0544 - val_mae: 0.7961
Epoch 5/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 599us/step - loss: 0.9737 - mae: 0.7704 - val_loss: 0.9817 - val_mae: 0.7716
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 605us/step - loss: 0.9236 - mae: 0.7545 - val_loss: 0.9470 - val_mae: 0.7592
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 791us/step - loss: 0.8960 - mae: 0.7438 - val_loss: 0.9323 - val_mae: 0.7548
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 2s 609us/step - loss: 0.8898 - mae: 0.7423 - val_loss: 0.9233 - val_mae: 0.7517
Epoch 9/10
2500/2500 ━━

In [19]:
class GAN():
    
    def __init__(self):
        self.latent_dim = latent_dim
        self.noise_dim = 100
        
        self.optimizer = Adam(0.0002, 0.5)
        
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=BinaryCrossentropy(),
            optimizer=self.optimizer,
            metrics=['accuracy'])

        self.generator = self.build_generator()

        z = Input(shape=(self.noise_dim,))
        fake_sample = self.generator(z)

        self.discriminator.trainable = False

        validity = self.discriminator(fake_sample)

        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=self.optimizer)
    
    def build_generator(self):
        model = Sequential([
            Dense(10, input_dim=self.noise_dim),
            LeakyReLU(alpha=0.2),
            BatchNormalization(momentum=0.8),
            Dense(20, input_dim=self.noise_dim),
            LeakyReLU(alpha=0.2),
            Dropout(rate=0.2),
            Dense(self.latent_dim * 2 + 1, activation='linear'),
        ])
        
        model.summary()
        
        noise = Input(shape=(self.noise_dim, ))
        sample = model(noise)
        
        return Model(noise, sample)
    
    def build_discriminator(self):
        model = Sequential([
            Dense(4, input_dim=self.latent_dim * 2 + 1),
            LeakyReLU(alpha=0.2),
            Dense(1, activation='sigmoid')
        ])
        
        model.summary()
        
        sample = Input(shape=(self.latent_dim * 2 + 1, ))
        validity = model(sample)
        
        return Model(sample, validity)
    
    def train(self, dataset, epochs, batch_size=128, sample_interval=50, num_training_samples=100000):
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        for epoch in range(epochs):
            
            idx = np.random.randint(0, num_training_samples, batch_size)
            real_samples = dataset[idx]
            
            noise = np.random.normal(0, 1, (batch_size, self.noise_dim))

            fake_samples = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(real_samples, valid)
            d_loss_fake = self.discriminator.train_on_batch(fake_samples, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            noise = np.random.normal(0, 1, (batch_size, self.noise_dim))

            g_loss = self.combined.train_on_batch(noise, valid)

            if epoch % sample_interval == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
                noise = np.random.normal(0, 1, (1, self.noise_dim))